### Imports

In [ ]:
# IMPORTS
import os
import time

import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
    TensorBoard,
)

from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns


### Dataset Setup

In [ ]:
#Dataset Path
dataset_path = r'.\dataset'
train_path = os.path.join(dataset_path, 'train_set')
dev_path = os.path.join(dataset_path, 'dev_set')
test_path = os.path.join(dataset_path, 'test_set')

#Hyperparameters
batch_size = 16
IMG_SIZE = (224, 224) # Resize all images to 224x224 as required by ResNet50

#Dataset split
train_set = image_dataset_from_directory(
    train_path,
    labels="inferred",
    label_mode = 'categorical',
    image_size=IMG_SIZE,
    batch_size=batch_size,
)

dev_set = image_dataset_from_directory(
    dev_path,
    labels="inferred",
    label_mode = 'categorical',
    image_size=IMG_SIZE,
    batch_size=batch_size,
)

test_set = image_dataset_from_directory(
    test_path,
    labels="inferred",
    label_mode = 'categorical',
    image_size=IMG_SIZE,
    batch_size=batch_size,
)

In [ ]:
# Visualize Data
class_names = train_set.class_names
plt.figure(figsize=(10, 10))
for images, labels in train_set.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis("off")

### Data Augmentation

In [ ]:
# Define Data Augmentation function
def data_augmenter():
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(tfl.RandomFlip('horizontal'))
    data_augmentation.add(tfl.RandomRotation(0.10))
    return data_augmentation

In [ ]:
# Visualize Data Augmentation
data_augmentation = data_augmenter()

for image, _ in train_set.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0), training = True)
        plt.imshow(augmented_image[0] / 255)
        plt.axis('off')

### Define Model

In [ ]:
############################################
#   - Define the ResNet50 base model, do not include top (prediction) layers, grab weights from imagenet dataset
#   - Freeze layers by setting base_model.trainable = False
#   - Define inputs
#   - Apply Data Augmentation
#   - Preproccess Inputs with ResNet50 preproccess utility
#   - Set Training = False for base_model in order not to update Batch Norm values
############################################

input_shape = IMG_SIZE + (3,)

# Define ResNet50 base model with weights from imagenet. Do NOT include top classification layers.
base_model = tf.keras.applications.resnet_v2.ResNet50V2(
    input_shape=input_shape, include_top=False, weights="imagenet"
)

# Freeze base model
base_model.trainable = False

# Define input layer
inputs = tf.keras.Input(shape=input_shape)

# Apply Data Augmentation
x = data_augmentation(inputs)

# Preproccess input using the same weights base model was trained on
x = tf.keras.applications.resnet_v2.preprocess_input(x)

# Set training = False to disable Batch Norm layers from updating
x = base_model(x, training=False)

# Add avaragePooling
x = tfl.GlobalAveragePooling2D()(x)

# Add dropout layer for regularization
x = tfl.Dropout(0.2)(x)

# Add prediction/output layer with 3 neurons (Class Number = 3)
outputs = tfl.Dense(3, activation='softmax', kernel_initializer=HeNormal())(x)

model = tf.keras.Model(inputs, outputs)


In [ ]:
# Print Model Summary
model.summary()

### Compile model

In [ ]:
# Define metrics
metrics = [
    tf.keras.metrics.TruePositives(name="tp"),
    tf.keras.metrics.FalsePositives(name="fp"),
    tf.keras.metrics.TrueNegatives(name="tn"),
    tf.keras.metrics.FalseNegatives(name="fn"),
    tf.keras.metrics.CategoricalAccuracy(name="Accuracy"),
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall"),
]


In [ ]:
# Compile model
model.compile(optimizer=Adam(learning_rate = 0.004), loss=CategoricalCrossentropy(), metrics=metrics)

### Callbacks

In [ ]:
# Checkpoint Directory setup
def create_checkpoint_dir():
    root_dir = os.path.join(os.curdir,"checkpoints","ResNet50")
    run_id = time.strftime('run_%d_%m_%Y-%H_%M_%S')
    checkpoint_logs = os.path.join(root_dir, run_id, "ckp_{epoch:02d}-{val_loss:.4f}")
    return checkpoint_logs

checkpoint_logs = create_checkpoint_dir() # .\checkpoints\ResNet50\run_22_07_2022-14_38_09\ckp_{epoch:02d}-{val_loss:.4f}
print(checkpoint_logs)

#Tensorboard log directory setup
def create_board_dir():
    root_dir = os.path.join(os.curdir,"board_logs","ResNet50")
    run_id = time.strftime('run_%d_%m_%Y-%H_%M_%S')
    logs = os.path.join(root_dir, run_id)
    return logs

logs = create_board_dir() # e.g., .\callbacks\board_logs\ResNet50\run_14-07-2022-15:14:09
print(logs)

In [ ]:
# Define callbacks
checkpoint_cb = ModelCheckpoint(checkpoint_logs, save_weights_only=True, save_best_only=True, monitor='val_loss', verbose=1),
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1),
reduce_LR_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1),
tensorboard_cb = TensorBoard(log_dir=logs,histogram_freq=1, write_images=True)

In [ ]:
callbacks = [checkpoint_cb,early_stopping_cb,reduce_LR_cb,tensorboard_cb]

### Train

In [ ]:
history = model.fit(train_set, validation_data = dev_set, epochs=100,verbose=1, callbacks=callbacks)

### Finetune

In [ ]:
base_model.trainable = True # unfreeze all model layers
base_model.summary() # Find a suitable layer name to fine-tune from
print("Number of layers in base model: ", len(base_model.layers))

In [ ]:
# Find layer number you want to fine-tune from
index = None
for idx, layer in enumerate(base_model.layers):
    if layer.name == 'conv4_block6_out': # Put layer name here
        index = idx
        break
print(idx)

# Make sure it's the correct index
print(base_model.layers[153])

In [ ]:
# Freeze all until 154
fine_tune_from = 154
for layer in base_model.layers[:fine_tune_from]:
    print('Layer ' + layer.name + ' frozen.')
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
# compile the model again in order to apply trainable layers change, with a lower learning rate
model.compile(optimizer=Adam(learning_rate = 0.00004), loss=CategoricalCrossentropy(), metrics=metrics)

In [ ]:
# Finetune model for 10 epochs
tuned_epochs = 10
total_epochs = len(history.epoch) + tuned_epochs
history_tuned = model.fit(train_set, validation_data = dev_set, initial_epoch=history.epoch[-1], epochs=total_epochs,verbose=1, callbacks=callbacks)

### Evaluate

In [ ]:
model.evaluate(test_set, batch_size=batch_size, verbose=1)

In [ ]:
predictions = np.array([])
labels =  np.array([])
for x, y in test_set:
  predictions = np.concatenate([predictions, np.argmax(model.predict(x), axis = -1)])
  labels = np.concatenate([labels, np.argmax(y.numpy(), axis=-1)])

### Metrics

In [ ]:
# Run Classification Report
print(classification_report(labels, predictions, target_names=['COVID-19', 'Non-COVID', 'Normal']))

In [ ]:
# Calcuate Confusion Matrix
cm = confusion_matrix(labels, predictions)

In [ ]:
# plot confusion matrix with seaborn
plt.figure(figsize=(10, 5))
group_counts = ["{0:0.0f}".format(value) for value in
                cm.flatten()]

group_percentages = ["{0:.2%}".format(value) for value in
                     cm.flatten()/np.sum(cm)]

cm_labels = [f"{v1}\n{v2}\n" for v1, v2 in
          zip(group_counts,group_percentages)]

cm_labels = np.asarray(cm_labels).reshape(3,3)

sns.set(font_scale=1.4)
ax = sns.heatmap(cm, annot=cm_labels, fmt='', linewidths=.5, cmap='YlGnBu')
ax.set_title('Confusion Matrix')
ax.set_xlabel('Predicted Values')
ax.set_ylabel('Actual values')

ax.xaxis.set_ticklabels(['COVID-19','Non-COVID19', 'Normal'])
ax.yaxis.set_ticklabels(['COVID-19','Non-COVID19', 'Normal'])
plt.show()